# C语言代码Bug检测演示

本notebook演示如何使用DSPy实现的代码bug检测功能。

## 1. 环境设置

首先，加载DSPy模型配置和bug检测模块。

In [2]:
# 导入必要的库
import dspy
import os
from code_bug_detector import CodeBugDetector, BUG_TYPE_MAPPING

In [3]:
# 配置DSPy模型 (与DPSy_tes.ipynb中的配置相同)
lm = dspy.LM(
    model='gemini/gemini-3-flash-preview',
    api_key=os.getenv('YAIzaSyCNwoqb59pZ49ef2ZCdmQrHw6nNKKQygdU'),
    max_tokens=4000
)

dspy.configure(lm=lm)
print("✓ DSPy模型配置完成")

✓ DSPy模型配置完成


In [4]:
# 创建bug检测器实例
detector = CodeBugDetector()
print("✓ Bug检测器已初始化")

✓ Bug检测器已初始化


## 2. 示例1：内存泄漏检测

In [5]:
code_example_1 = """
void process_data() {
    int* data = (int*)malloc(100 * sizeof(int));
    if (data == NULL) {
        return;
    }
    
    // 处理数据
    for (int i = 0; i < 100; i++) {
        data[i] = i * 2;
    }
    
    // 忘记释放内存！
}
"""

print("检测代码片段 1：")
print(code_example_1)
print("\n开始检测...")

result1 = detector(code_example_1)
print(detector.format_output(result1))

检测代码片段 1：

void process_data() {
    int* data = (int*)malloc(100 * sizeof(int));
    if (data == NULL) {
        return;
    }

    // 处理数据
    for (int i = 0; i < 100; i++) {
        data[i] = i * 2;
    }

    // 忘记释放内存！
}


开始检测...


2026/01/17 20:17:56 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


AuthenticationError: litellm.AuthenticationError: GeminiException - {
  "error": {
    "code": 400,
    "message": "API key not valid. Please pass a valid API key.",
    "status": "INVALID_ARGUMENT",
    "details": [
      {
        "@type": "type.googleapis.com/google.rpc.ErrorInfo",
        "reason": "API_KEY_INVALID",
        "domain": "googleapis.com",
        "metadata": {
          "service": "generativelanguage.googleapis.com"
        }
      },
      {
        "@type": "type.googleapis.com/google.rpc.LocalizedMessage",
        "locale": "en-US",
        "message": "API key not valid. Please pass a valid API key."
      }
    ]
  }
}


## 3. 示例2：缓冲区溢出

In [ ]:
code_example_2 = """
void copy_string(char* input) {
    char buffer[10];
    strcpy(buffer, input);  // 没有检查input的长度！
    printf("Copied: %s\n", buffer);
}
"""

print("检测代码片段 2：")
print(code_example_2)
print("\n开始检测...")

result2 = detector(code_example_2)
print(detector.format_output(result2))

## 4. 示例3：空指针解引用

In [ ]:
code_example_3 = """
int get_value(int* ptr) {
    return *ptr;  // 没有检查ptr是否为NULL！
}

int main() {
    int* p = NULL;
    int value = get_value(p);
    printf("Value: %d\n", value);
    return 0;
}
"""

print("检测代码片段 3：")
print(code_example_3)
print("\n开始检测...")

result3 = detector(code_example_3)
print(detector.format_output(result3))

## 5. 示例4：数组越界

In [ ]:
code_example_4 = """
void fill_array() {
    int arr[10];
    for (int i = 0; i <= 10; i++) {  // off-by-one错误！
        arr[i] = i;
    }
}
"""

print("检测代码片段 4：")
print(code_example_4)
print("\n开始检测...")

result4 = detector(code_example_4)
print(detector.format_output(result4))

## 6. 示例5：正确的代码（无bug）

In [ ]:
code_example_5 = """
int safe_add(int a, int b, int* result) {
    if (result == NULL) {
        return -1;  // 错误：空指针
    }
    
    // 检查整数溢出
    if (a > 0 && b > INT_MAX - a) {
        return -1;  // 错误：溢出
    }
    if (a < 0 && b < INT_MIN - a) {
        return -1;  // 错误：下溢
    }
    
    *result = a + b;
    return 0;  // 成功
}
"""

print("检测代码片段 5：")
print(code_example_5)
print("\n开始检测...")

result5 = detector(code_example_5)
print(detector.format_output(result5))

## 7. 自定义代码检测

你可以在下面的单元格中输入自己的C代码进行检测。

In [ ]:
# 在这里输入你要检测的C代码
custom_code = """
// 在这里输入你的C代码
"""

if custom_code.strip() and "在这里输入" not in custom_code:
    print("检测自定义代码：")
    print(custom_code)
    print("\n开始检测...")
    
    custom_result = detector(custom_code)
    print(detector.format_output(custom_result))
else:
    print("请先输入要检测的代码")

## 8. 查看支持的Bug类型

查看系统支持的所有bug类型及其描述。

In [ ]:
print("支持的Bug类型列表：")
print("=" * 80)
for bug_id, description in BUG_TYPE_MAPPING.items():
    print(f"{bug_id}: {description}")
print("=" * 80)

## 9. 批量检测

如果你有多个代码片段需要检测，可以使用批量检测功能。

In [ ]:
# 示例：批量检测多个代码片段
code_samples = [
    code_example_1,
    code_example_2,
    code_example_3,
    code_example_4,
    code_example_5
]

print("开始批量检测...\n")

for i, code in enumerate(code_samples, 1):
    print(f"\n{'='*80}")
    print(f"代码片段 #{i}")
    print(f"{'='*80}")
    
    result = detector(code)
    print(detector.format_output(result))
    
print("\n批量检测完成！")

## 10. 检测结果统计

统计检测到的bug类型分布。

In [ ]:
from collections import Counter

def analyze_detection_results(results):
    """
    分析多个检测结果，统计bug类型分布
    """
    bug_types = []
    total_bugs = 0
    
    for result in results:
        if result['has_bug']:
            bugs = result['bug_details'].get('bugs', [])
            for bug in bugs:
                bug_type_id = bug.get('bug_type_id', 'BT020')
                bug_types.append(bug_type_id)
                total_bugs += 1
    
    # 统计每种bug类型的数量
    bug_counter = Counter(bug_types)
    
    print("Bug类型分布统计：")
    print("=" * 80)
    print(f"总计检测到 {total_bugs} 个bug")
    print("\n各类型bug数量：")
    for bug_id, count in bug_counter.most_common():
        print(f"  {bug_id} ({BUG_TYPE_MAPPING.get(bug_id, 'Unknown')}): {count}")
    print("=" * 80)

# 示例使用（取消注释以运行）
# results = [result1, result2, result3, result4, result5]
# analyze_detection_results(results)